# Custom Climate Profiles Generation


This notebook generates custom `Climate Profile` hourly datasets (8760) of two varieties. 

1. Standard Year Profile (single variable) -- *in development*
2. Typical Meteorological Year Profile (TMY, multi-variable)

**Coming soon!** This work is *in development* by the AE team. 

### What is a climate profile?


### 2 Different kinds of profiles

1. AMY
2. TMY

Are you just looking to generate profiles for one of the pre-generated 32 weather stations? Check out this page.

## Setup

In [2]:
%config InlineBackend.figure_format = 'svg' # Make plots look better in the notebook environment 
%reload_ext autoreload
%autoreload 2

from climakitae.explore.amy import get_climate_profile

import climakitae as ck
import pandas as pd

## Average Meteoroligical Year

Examining a particular month within the Average Meteorological Year can provide hourly information that could inform energy usage analysis. For example, a warm season month can be used to examine cooling demand; while a cold season month can be used for exploring heating demand change under future climate conditions.

To analyze a specific month, we'll retrieve some of the absolute AMY data of interest shown above. Please select in the explore panel above 'Absolute' as your AMY Type, and either 'Historical' or 'Warming Level Future' with a warming level of your choosing. A progress bar is provided when running the cell below, as this step may take a little while. 

Visualize one month from the Average Meteorological Year 
First, we'll subset the dataframe to grab data for the month of January. You can change the month easily by changing the variable `month`; just make sure the value you is the shortened version of the month (i.e. "Jan" instead of "January") so that the code can find the substring in the `amy_data` object. 

https://github.com/cal-adapt/climakitae/blob/8182900aa62947285cb8ebbe6f29d9b9acdc6991/climakitae/explore/amy.py#L214

### Generating AMY

        - variable (Optional) : str, default "Air Temperature at 2m"
        - resolution (Optional) : str, default "3 km"
        - warming_level (Required) : List[float], default [1.2]
        - cached_area (Optional) : str or List[str]
        - units (Optional) : str, default "degF"
        - latitude (Optional) : float or tuple
        - longitude (Optional) : float or tuple
        - stations (Optional) : list[str], default None
        - days_in_year (Optional) : int, default 365
        - q (Optional) : float | list[float], default 0.5, quantile for profile calculation
        - no_delta (optional) : bool, default False, if True, do not apply baseline subtraction, return raw future profile


In [ ]:
variable =  # str, default "Air Temperature at 2m"
resolution = # str, default "3 km"
warming_level =  # List[float], default [1.2]
cached_area =  # str or List[str]
units = # str, default "degF"
latitude = # float or tuple
longitude = # float or tuple
stations = # list[str], default None
days_in_year = # : int, default 365
q = # float | list[float], default 0.5, quantile for profile calculation
no_delta = # bool, default False, if True, do not apply baseline subtraction, return raw future profile


In [3]:
selection = {
    "variable": "Air Temperature at 2m",
    "resolution": "3 km",
    "warming_level": [2.0],
    # "warming_level": [1.5, 1.75, 2.0, 2.25, 2.5],
    "units": "degF",
    "cached_area": ["Los Angeles County"],
}

In [ ]:
profile = get_climate_profile(**selection)

📊 Retrieving climate data...


Data retrieval:   0%|          | 0/2 [00:00<?, ?dataset/s]

⚙️  Computing climate profiles...
      📊 Processing 262,800 hours (30 years) of data
      🎯 Computing 50th percentile for each hour of year
      ⚙️ Computing quantiles for 1 warming level(s) and 8 simulation(s)


      Computing profiles:   0%|          | 0/8 [00:00<?, ?combo/s]

      ✅ Profile computation complete! Final shape: (365, 192)
         With index: Day of Year, columns: ['Hour', 'Simulation']
         Units: degF
      📊 Processing 262,800 hours (30 years) of data
      🎯 Computing 50th percentile for each hour of year
      ⚙️ Computing quantiles for 1 warming level(s) and 8 simulation(s)


      Computing profiles:   0%|          | 0/8 [00:00<?, ?combo/s]

###  Visualization

import matplotlib.pyplot as plt
import pandas as pd

# plotting for only one warming level
idx = pd.IndexSlice
sims = profile.columns.get_level_values("Simulation").unique().tolist()

for sim in sims:
    # Select columns for the current simulation
    data = profile.loc[:, idx[:, sim]]
    # Assuming the first level is month and the second is simulation
    # We'll plot a heatmap of the data for each simulation
    plt.figure(figsize=(12, 6))
    plt.imshow(data.values.T, aspect="auto", cmap="coolwarm")
    plt.colorbar(label=f"{selection['variable']} ({selection['units']})")
    plt.yticks(range(data.shape[1]), data.columns.get_level_values(0))
    plt.xticks(range(data.shape[0]), data.index)
    plt.title(
        f"{sim} - {selection['variable']} at {selection['warming_level'][0]}°C Warming"
    )
    plt.xlabel("Day of Year")
    plt.ylabel("Hour of Day")
    vmin = -max(abs(data.values.max()), abs(data.values.min()))
    vmax = max(abs(data.values.max()), abs(data.values.min()))
    xtick_freq = max(1, data.shape[0] // 12)
    plt.xticks(range(0, data.shape[0], xtick_freq), data.index[::xtick_freq])
    plt.clim(vmin, vmax)
    plt.show()